In [6]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
    adjusted_rand_score
)

from sklearn.decomposition import PCA


# Пути и общие настройки

DATA_DIR = Path("data")
ARTIFACTS_DIR = Path("artifacts")
FIGURES_DIR = ARTIFACTS_DIR / "figures"
LABELS_DIR = ARTIFACTS_DIR / "labels"

FIGURES_DIR.mkdir(parents=True, exist_ok=True)
LABELS_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_STATE = 42

datasets = {
    "ds1": "S07-hw-dataset-01.csv",
    "ds2": "S07-hw-dataset-02.csv",
    "ds4": "S07-hw-dataset-04.csv",
}

metrics_summary = {}
best_configs = {}


# Вспомогательная функция для расчёта метрик

def calc_metrics(X, labels):
    return {
        "silhouette": float(silhouette_score(X, labels)),
        "davies_bouldin": float(davies_bouldin_score(X, labels)),
        "calinski_harabasz": float(calinski_harabasz_score(X, labels)),
    }


# Основной цикл по датасетам

for name, fname in datasets.items():
    print(f"\n================ {name.upper()} =================")
    df = pd.read_csv(DATA_DIR / fname)

    print("Первые строки датасета:")
    print(df.head())

    print("\nИнформация о данных:")
    print(df.info())

    sample_id = df["sample_id"]
    X = df.drop(columns=["sample_id"])

    
    # Определяем типы признаков
    
    num_cols = X.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = X.select_dtypes(exclude=["number"]).columns.tolist()

    print(f"Числовые признаки: {len(num_cols)}")
    print(f"Категориальные признаки: {len(cat_cols)}")

    
    # Препроцессинг
    
    transformers = []

    if num_cols:
        transformers.append((
            "num",
            Pipeline([
                ("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler())
            ]),
            num_cols
        ))

    if cat_cols:
        transformers.append((
            "cat",
            Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore"))
            ]),
            cat_cols
        ))

    preprocessor = ColumnTransformer(transformers)
    X_prep = preprocessor.fit_transform(X)

    
    # KMeans — подбор k по silhouette
    
    print("\nПодбор числа кластеров для KMeans...")
    sil_scores = {}

    for k in range(2, 11):
        km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
        labels = km.fit_predict(X_prep)
        sil_scores[k] = silhouette_score(X_prep, labels)

    best_k = max(sil_scores, key=sil_scores.get)
    print(f"Лучшее k для KMeans: {best_k}")

    plt.figure()
    plt.plot(list(sil_scores.keys()), list(sil_scores.values()), marker="o")
    plt.xlabel("k")
    plt.ylabel("Silhouette")
    plt.title(f"{name}: Silhouette vs k (KMeans)")
    plt.savefig(FIGURES_DIR / f"{name}_kmeans_silhouette_k.png")
    plt.close()

    kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=10)
    km_labels = kmeans.fit_predict(X_prep)


# DBSCAN — подбор eps (с проверкой числа кластеров)

print("\nПодбор параметра eps для DBSCAN...")
best_dbscan = None
best_db_sil = -1

for eps in [0.3, 0.5, 0.7, 1.0]:
    db = DBSCAN(eps=eps, min_samples=5)
    labels = db.fit_predict(X_prep)

    noise_ratio = (labels == -1).mean()

    # оставляем только non-noise 
    core_labels = labels[labels != -1]
    core_points = X_prep[labels != -1]

    # количество кластеров (без шума)
    n_clusters = len(set(core_labels))

    print(
        f"eps={eps}: доля шума={noise_ratio:.3f}, "
        f"кластеров (без шума)={n_clusters}"
    )

    # silhouette корректен только при >= 2 
    if n_clusters < 2:
        print("  Silhouette не считается (менее 2 кластеров).")
        continue

    sil = silhouette_score(core_points, core_labels)

    if sil > best_db_sil:
        best_db_sil = sil
        best_dbscan = (db, labels, eps, noise_ratio)

if best_dbscan is not None:
    db_model, db_labels, best_eps, noise_ratio = best_dbscan
    print(f"Лучший eps для DBSCAN: {best_eps}")
    print(f"Доля шума: {noise_ratio:.3f}")
    print(
        "Метрики DBSCAN рассчитаны только по non-noise точкам "
        "(label != -1)."
    )
else:
    print("DBSCAN не дал устойчивого разбиения (>=2 кластеров).")
    db_labels = None
    noise_ratio = 1.0

    
    # Agglomerative Clustering
    
    print("\nЗапуск AgglomerativeClustering...")
    agg = AgglomerativeClustering(n_clusters=best_k, linkage="ward")
    agg_labels = agg.fit_predict(X_prep)

    
    # Метрики качества
    
    metrics_summary[name] = {
        "kmeans": calc_metrics(X_prep, km_labels),
        "agglomerative": calc_metrics(X_prep, agg_labels),
        "dbscan": (
            calc_metrics(
                X_prep[db_labels != -1],
                db_labels[db_labels != -1]
            ) if db_labels is not None and noise_ratio < 0.9 else None
        ),
        "dbscan_noise_ratio": float(noise_ratio)
    }

    
    # PCA (2D) — только для визуализации
    
    pca = PCA(n_components=2, random_state=RANDOM_STATE)
    X_pca = pca.fit_transform(
        X_prep.toarray() if hasattr(X_prep, "toarray") else X_prep
    )

    plt.figure()
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=km_labels, s=10)
    plt.title(f"{name}: PCA (2D проекция, KMeans)")
    plt.savefig(FIGURES_DIR / f"{name}_pca.png")
    plt.close()

    
    # Сохранение меток кластеров (лучшее решение)
    
    out = pd.DataFrame({
        "sample_id": sample_id,
        "cluster_label": km_labels
    })
    out.to_csv(LABELS_DIR / f"labels_hw07_{name}.csv", index=False)

    best_configs[name] = {
        "method": "KMeans",
        "k": best_k
    }


# Проверка устойчивости (ds1)

print("\nПроверка устойчивости KMeans на ds1...")

df = pd.read_csv(DATA_DIR / datasets["ds1"])
X = df.drop(columns=["sample_id"])

# Масштабирование (как и в основном эксперименте)
X = StandardScaler().fit_transform(X)

# Явно фиксируем k (из основного эксперимента для ds1)
k_stability = 2

labels_list = []
for seed in range(5):
    km = KMeans(
        n_clusters=k_stability,
        random_state=seed,
        n_init=10
    )
    labels_list.append(km.fit_predict(X))

ari_scores = [
    adjusted_rand_score(labels_list[0], labels_list[i])
    for i in range(1, 5)
]

print("ARI между запусками:", ari_scores)


# Сохранение артефактов

with open(ARTIFACTS_DIR / "metrics_summary.json", "w", encoding="utf-8") as f:
    json.dump(metrics_summary, f, indent=2, ensure_ascii=False)

with open(ARTIFACTS_DIR / "best_configs.json", "w", encoding="utf-8") as f:
    json.dump(best_configs, f, indent=2, ensure_ascii=False)




================ DS1 =================
Первые строки датасета:
   sample_id        f01        f02       f03         f04        f05  \
0          0  -0.536647 -69.812900 -0.002657   71.743147 -11.396498   
1          1  15.230731  52.727216 -1.273634 -104.123302  11.589643   
2          2  18.542693  77.317150 -1.321686 -111.946636  10.254346   
3          3 -12.538905 -41.709458  0.146474   16.322124   1.391137   
4          4  -6.903056  61.833444 -0.022466  -42.631335   3.107154   

         f06        f07       f08  
0 -12.291287  -6.836847 -0.504094  
1  34.316967 -49.468873  0.390356  
2  25.892951  44.595250  0.325893  
3   2.014316 -39.930582  0.139297  
4  -5.471054   7.001149  0.131213  

Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sample_id  12000 non-null  int64  
 1   f01        12000 non-null  float64
 2 